In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from typing import List
from datasetsforecast.long_horizon import LongHorizon
import pandas as pd
import numpy as np
import torch
from typing import Optional

import os
import pathlib

from torch.utils.data import Dataset, DataLoader, Subset
from tqdm.auto import tqdm

from dataset import LongHorizonUnivariateDataModule, LongHorizonUnivariateDataset
from dataset import ElectricityUnivariateDataModule, ElectricityUnivariateDataset
import pytorch_lightning as pl


import bisect

/opt/conda/lib/python3.10/site-packages/lightning_fabric/__init__.py:29: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)


In [3]:
mhlv = pd.read_csv('data/electricity/datasets/mhlv/raw/MHLV.csv')
mhlv = mhlv.set_index('Datetime')
mhlv

FileNotFoundError: [Errno 2] No such file or directory: 'data/electricity/datasets/mhlv/raw/MHLV.csv'

In [ ]:
mhlv_test = pd.read_csv('data/electricity/datasets/mhlv/raw/MHL_test2.csv', 
                        header=None, names=mhlv.columns)
mhlv_test.set_index(mhlv.index[-len(mhlv_test):], inplace=True)
mhlv_test

In [ ]:
np.isclose(mhlv_test, mhlv[-len(mhlv_test):]).sum(axis=0)

In [ ]:
mhlv[-len(mhlv_test):] = (mhlv[-len(mhlv_test):] + mhlv_test) / 2
mhlv[-len(mhlv_test):]

In [ ]:
np.isclose(mhlv_test, mhlv[-len(mhlv_test):]).sum(axis=0)

In [ ]:
mhlv = mhlv.stack(dropna=False).reset_index()
mhlv.rename({'Datetime': 'ds', 'level_1': 'unique_id', 0: 'y'}, inplace=True, axis=1)
mhlv

In [ ]:
rm data/electricity/datasets/mhlv.zip

In [ ]:
mhlv_path = 'data/electricity/datasets/mhlv/M'

pathlib.Path(mhlv_path).mkdir(parents=True, exist_ok=True)
mhlv.to_csv(os.path.join(mhlv_path, 'df_y.csv'), index=False)

!zip -r data/electricity/datasets/mhlv.zip data/electricity/datasets/mhlv # -x "data/electricity/datasets/mhlv/raw/*"

In [ ]:
# import subprocess

# proc = subprocess.run(["gsutil", "-m", "rsync", "data/electricity/datasets/", "gs://electricity-datasets"],
#                       capture_output=True)
# print(proc.stderr.decode())

In [ ]:
# !rm data/electricity/datasets/mhlv.zip
# !rm -rf data/electricity/datasets/mhlv/M
# !gsutil -m rsync -r gs://electricity-datasets data/electricity/datasets/

In [ ]:
ElectricityUnivariateDataModule.prepare_data()

# Test the download

In [ ]:
path = os.path.join('./data/electricity/datasets', 'mhlv', 'M/df_y.csv')
df = pd.read_csv(path)
df[-35:]

In [ ]:
mhlv_test_ds = ElectricityUnivariateDataset(name='mhlv', split='train', split_start='2018-01-01', split_end='2019-01-01')


In [ ]:
mhlv_test_ds.df[mhlv_test_ds.history_length:]

In [ ]:
ls data/electricity/datasets/

In [ ]:
# !zip -r data/electricity/datasets/mhlv.zip data/electricity/datasets/mhlv

In [ ]:
# mv data/electricity/datasets/mhlv data/electricity/datasets/mhlv-old

In [ ]:
# !unzip data/electricity/datasets/mhlv.zip

In [ ]:
!rm data/electricity/datasets/mhlv.zip
!rm -r data/electricity/datasets/mhlv
# !ls data/electricity/datasets/

In [ ]:
from glob import glob

print("Downloading datasets")
datasets_path = "data/electricity/datasets/"
proc = subprocess.run(["gsutil", "-m", "rsync", "gs://electricity-datasets", datasets_path],
                      capture_output=True)
print(proc.stderr.decode())
datasets = glob(os.path.join(datasets_path, "*.zip"))

print("Unzipping datasets")
for d in datasets:
    proc = subprocess.run(["unzip", "-u", d], capture_output=True)
    print(proc.stdout.decode())

In [ ]:
mhlv = ElectricityUnivariateDataset(name='mhlv', split='train', split_start=0, split_end=0.8, step=10)
self = mhlv

In [ ]:
self.num_windows = (~self.df.isnull()).sum(axis=0) - self.tot_window_len + 1

self.df.drop(self.df.columns[self.num_windows <= 0], axis=1, inplace=True)
self.num_windows = (~self.df.isnull()).sum(axis=0) - self.tot_window_len + 1
self.cum_num_windows = self.num_windows.cumsum()
self.num_nulls = self.df.isnull().sum(axis=0)


In [ ]:
index=54658-1

index = 259139 - 5467

column_idx = bisect.bisect_right(self.cum_num_windows, index)
window_idx = self.num_nulls[column_idx] + self.step * index - (self.step * self.cum_num_windows[column_idx-1] if column_idx > 0 else 0)

print('column_idx', column_idx)
print('window_idx', window_idx)

In [ ]:
self.df.values[window_idx:window_idx+self.tot_window_len, column_idx].astype(int)[:20]

In [ ]:
self.df.values[window_idx:window_idx+self.tot_window_len, column_idx].astype(int)[:20]

In [ ]:
len(self.df.values[89720:, 0])

In [ ]:
len(self.df)

In [ ]:
mhlv.cum_num_windows

In [ ]:
self.df.iloc[:,column_idx][:500]